In [6]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import psycopg2
import logging

In [7]:
def create_tables(engine, logging):
    with engine.connect() as connection:
        try:
            connection.execute(text("""DROP TABLE IF EXISTS Restaurant, Cuisine, Meal, Purpose, Restaurant_Cuisine, Restaurant_Meal, Restaurant_Purpose CASCADE"""))
            
            connection.execute(text("""
                CREATE TABLE IF NOT EXISTS Restaurant (
                    id SERIAL PRIMARY KEY,
                    city varchar(255) NOT NULL,
                    link varchar(255) NOT NULL,
                    name varchar(255) NOT NULL,
                    rating float8 DEFAULT -1,
                    price_lower float8 DEFAULT -1.0,
                    price_higher float8 DEFAULT -1.0,
                    price_type varchar(255),
                    longitude float8 DEFAULT -1.0,
                    latitude float8 DEFAULT -1.0
                )
            """))

            connection.execute(text("""
                CREATE TABLE IF NOT EXISTS Cuisine (
                    id SERIAL PRIMARY KEY,
                    name varchar(255) NOT NULL
                )
            """))
            
            connection.execute(text("""
                CREATE TABLE IF NOT EXISTS Meal (
                    id SERIAL PRIMARY KEY,
                    name varchar(255) NOT NULL
                )
            """))
            
            connection.execute(text("""
                CREATE TABLE IF NOT EXISTS Purpose (
                    id SERIAL PRIMARY KEY,
                    name varchar(255) NOT NULL
                )
            """))


            connection.execute(text("""
                CREATE TABLE IF NOT EXISTS Restaurant_Cuisine (
                    restaurant_id INTEGER,
                    cuisine_id INTEGER,
                    PRIMARY KEY (restaurant_id, cuisine_id),
                    FOREIGN KEY (restaurant_id) REFERENCES Restaurant(id),
                    FOREIGN KEY (cuisine_id) REFERENCES Cuisine(id)
                )
            """))
            
            connection.execute(text("""
                CREATE TABLE IF NOT EXISTS Restaurant_Meal (
                    restaurant_id INTEGER,
                    meal_id INTEGER,
                    PRIMARY KEY (restaurant_id, meal_id),
                    FOREIGN KEY (restaurant_id) REFERENCES Restaurant(id),
                    FOREIGN KEY (meal_id) REFERENCES Meal(id)
                )
            """))
            
            connection.execute(text("""
                CREATE TABLE IF NOT EXISTS Restaurant_Purpose (
                    restaurant_id INTEGER,
                    purpose_id INTEGER,
                    PRIMARY KEY (restaurant_id, purpose_id),
                    FOREIGN KEY (restaurant_id) REFERENCES Restaurant(id),
                    FOREIGN KEY (purpose_id) REFERENCES Purpose(id)
                )
            """))
            
            connection.commit()
            print("Table creation successful!")
        except Exception as e:
            logging.error(f"Error during creating tables: {e}")
            print("Error during creating tables, see the insert_data.log file")
            connection.rollback()

In [8]:
def insert_data(engine, data, logging):
    cuisine_id_mapping = {}
    meal_id_mapping = {}
    purpose_id_mapping = {}
    
    with engine.connect() as connection:
        try:
            for _, row in data.iterrows():
                curr_params = {"c":row['city'], "l":row['link'], "n":row['name'], "r":row['rating'],
                               "pl":row['price_lower'], "ph":row['price_higher'], "pt": row['price_type'] ,"lon":row['longitude'], "lat":row['latitude']}

                stmt = text("""INSERT INTO Restaurant(city, link, name, rating, price_lower, price_higher, price_type, longitude, latitude) VALUES(:c, :l, :n, :r, :pl, :ph, :pt, :lon, :lat) RETURNING id""")

                result = connection.execute(stmt.bindparams(**curr_params))

                restaurant_id = result.fetchone()[0]
                
                cuisines = [c.strip() for c in str(row['cuisine']).split(',') if c.strip() != '']
                for cuisine in cuisines:
                    if cuisine not in cuisine_id_mapping:
                        stmt_cuisine = text("""INSERT INTO Cuisine (name) VALUES (:cuisine) RETURNING id""")
                        cur_cuisine = {"cuisine":cuisine}
                        result = connection.execute(stmt_cuisine.bindparams(**cur_cuisine))
                        cuisine_id_mapping[cuisine] = result.fetchone()[0]

                    cuisine_id = cuisine_id_mapping[cuisine]

                    stmt_relation = text("""INSERT INTO Restaurant_Cuisine (restaurant_id, cuisine_id) VALUES (:rid, :cid)""")
                    curr_ids = {"rid":restaurant_id, "cid":cuisine_id}

                    existing_stmt = text("""SELECT 1 FROM Restaurant_Cuisine WHERE restaurant_id = :rid AND cuisine_id = :cid""")
                    existing_record = connection.execute(existing_stmt.bindparams(**curr_ids)).fetchone()

                    if not existing_record:
                        connection.execute(stmt_relation.bindparams(**curr_ids))
                
                meals = [c.strip() for c in str(row['meal_type']).split(',') if c.strip() != '']
                for meal in meals:
                    if meal not in meal_id_mapping:
                        stmt_meal = text("""INSERT INTO Meal (name) VALUES (:meal) RETURNING id""")
                        curr_meal = {"meal":meal}
                        result = connection.execute(stmt_meal.bindparams(**curr_meal))
                        meal_id_mapping[meal] = result.fetchone()[0]

                    meal_id = meal_id_mapping[meal]

                    stmt_relation = text("""INSERT INTO Restaurant_Meal (restaurant_id, meal_id) VALUES (:rid, :mid)""")
                    curr_ids = {"rid":restaurant_id, "mid":meal_id}

                    existing_stmt = text("""SELECT 1 FROM Restaurant_Meal WHERE restaurant_id = :rid AND meal_id = :mid""")
                    existing_record = connection.execute(existing_stmt.bindparams(**curr_ids)).fetchone()

                    if not existing_record:
                        connection.execute(stmt_relation.bindparams(**curr_ids))
                        
                purposes = [c.strip() for c in str(row['purpose']).split(',') if c.strip() != '']
                for purpose in purposes:
                    if purpose not in purpose_id_mapping:
                        stmt_purpose = text("""INSERT INTO Purpose (name) VALUES (:purpose) RETURNING id""")
                        curr_purpose = {"purpose":purpose}
                        result = connection.execute(stmt_purpose.bindparams(**curr_purpose))
                        purpose_id_mapping[purpose] = result.fetchone()[0]

                    purpose_id = purpose_id_mapping[purpose]

                    stmt_relation = text("""INSERT INTO Restaurant_Purpose (restaurant_id, purpose_id) VALUES (:rid, :pid)""")
                    curr_ids = {"rid":restaurant_id, "pid":purpose_id}

                    existing_stmt = text("""SELECT 1 FROM Restaurant_Purpose WHERE restaurant_id = :rid AND purpose_id = :pid""")
                    existing_record = connection.execute(existing_stmt.bindparams(**curr_ids)).fetchone()

                    if not existing_record:
                        connection.execute(stmt_relation.bindparams(**curr_ids))
                
            connection.commit()
            print("Data insertion successful!")
            
        except Exception as e:
            logging.error(f"Error during data insertion: {e}")
            print("Error during data insertion, see the insert_data.log file")
            connection.rollback()

        finally:
            connection.close()

In [10]:
csv_file_path = r'<csv_path>'
postgresql_url = 'postgresql://<username>:<password>@<host>:<port>/<db_name>'
logging.basicConfig(filename='insert_data.log', level=logging.ERROR, format='%(asctime)s - %(levelname)s: %(message)s')
    
data = pd.read_csv(csv_file_path, encoding='utf-8-sig')

engine = create_engine(postgresql_url)

create_tables(engine, logging)

insert_data(engine, data, logging)

Table creation successful!
Data insertion successful!
